<a href="https://colab.research.google.com/github/Kaushikpandav/diff2lip/blob/main/Copy_of_Diff2Lip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Demo for "[Diff2Lip: Audio Conditioned Diffusion Models for Lip-Synchronization](https://soumik-kanad.github.io/diff2lip/)"
- Colab Notebook Author: Soumik Mukhopadhyay (soumik@umd.edu)

## Instructions:
- Change the runtime to GPU from top menu:  Runtime > Change runtime type > Hardware accelarator > T4 GPU.
- If you are not using google chrome, you may need to manually upload the videos (using the left side bar) and set the path manually in the "Set these paths manually if you upload your files manually" cell.
  - You can download the example video from [here](https://drive.google.com/file/d/1yhsa3WQZS22s4uHTwma2t0Q48ZYD5BzR/view?usp=sharing).


In [ ]:
!nvidia-smi

Wed Sep 25 14:37:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title Clone repo
!git clone https://github.com/soumik-kanad/diff2lip.git

Cloning into 'diff2lip'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 93 (delta 17), reused 66 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 15.74 MiB | 26.03 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
#@title Install requirements
import os
os.chdir('diff2lip')
!pip install -r requirements.txt



Obtaining file:///content/diff2lip/guided-diffusion (from -r requirements.txt (line 11))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 88.6 MB/s eta 0:00:00
   ━━━━

In [ ]:
#@title Download Checkpoint
! mkdir -p checkpoints
! gdown 15vMcyPtFDEttNwelP3IouwsMpnccTqit -O checkpoints/checkpoint.pt

Downloading...
From (original): https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit
From (redirected): https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit&confirm=t&uuid=2bed4a1e-208a-4627-86e3-f4cb23a70f2c
To: /content/diff2lip/checkpoints/checkpoint.pt
100% 407M/407M [00:02<00:00, 144MB/s]


In [ ]:
#@title helper functions
from google.colab import files
import shutil
os.makedirs("data", exist_ok=True)

def upload_file():
    uploaded = files.upload()
    uploaded_paths = []
    for filename in uploaded.keys():
        dst_path = os.path.join("data", filename)
        print(f'move {filename} to {dst_path}')
        shutil.move(filename, dst_path)
        uploaded_paths.append(dst_path)
    return uploaded_paths

In [ ]:
#@title Upload Video
v = upload_file()
video_path=v[0]
audio_path=v[0]

Saving P5.mp4 to P5.mp4
move P5.mp4 to data/P5.mp4


In [ ]:
#@title Upload Audio (Upload again or skip this cell if the audio and video are present in the same video uploaded above)
a = upload_file()
audio_path=a[0]

In [ ]:
#@title Set these paths manually if you upload your files manually
# video_path="data/Harry_potter_hp_spanish_trim.mp4"
# audio_path="data/Harry_potter_hp_spanish_trim.mp4"
out_path="output.mp4"

In [ ]:
#@title Setting Flags
sample_mode="cross" # or "reconstruction"
generate_from_filelist=0
model_path="checkpoints/checkpoint.pt"
pads="0,0,0,0"

#cross vs reconstruction
if sample_mode == "reconstruction":
    sample_input_flags="--sampling_input_type=first_frame --sampling_ref_type=first_frame"
elif sample_mode == "cross":
    sample_input_flags="--sampling_input_type=gt --sampling_ref_type=gt"
else:
    print('Error: sample_mode can only be \"cross\" or \"reconstruction\"')

MODEL_FLAGS=f"--attention_resolutions 32,16,8 --class_cond False --learn_sigma True --num_channels 128 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm False"
DIFFUSION_FLAGS=f"--predict_xstart False  --diffusion_steps 1000 --noise_schedule linear --rescale_timesteps False"
SAMPLE_FLAGS=f"--sampling_seed=7   {sample_input_flags} --timestep_respacing ddim25 --use_ddim True --model_path={model_path}"
DATA_FLAGS=f"--nframes 5 --nrefer 1 --image_size 128 --sampling_batch_size=32 "
TFG_FLAGS=f"--face_hide_percentage 0.5 --use_ref=True --use_audio=True --audio_as_style=True"
GEN_FLAGS=f"--generate_from_filelist {generate_from_filelist}  --video_path={video_path} --audio_path={audio_path} --out_path={out_path} --save_orig=False --face_det_batch_size 16 --pads {pads} --is_voxceleb2=False"



In [ ]:
#@title Run Generation (took ~ 4 mins for the example video)
!python generate.py {MODEL_FLAGS}  {DIFFUSION_FLAGS}  {SAMPLE_FLAGS} {DATA_FLAGS} {TFG_FLAGS} {GEN_FLAGS}

2024-09-25 16:21:34.563450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 16:21:34.597900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 16:21:34.607692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 16:21:34.631528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 16:21:36.477272: W tensorflow/compiler/tf2

In [ ]:
#@title Display Output Video
import moviepy.editor
moviepy.editor.ipython_display("output.mp4")

In [ ]:
#@title Step1: Setup Wav2Lip
#@markdown * Install dependency
#@markdown * Download pretrained model
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")




Done


In [ ]:

%cd sample_data/
%rm input_audio.wav
%rm input_video.mp4
from google.colab import files
uploaded = files.upload()
%cd ..
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4')
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
""")

In [ ]:

#@title 2.Use resize_factor to reduce the video resolution, as there is a change you might get better results for lower resolution videos. Why? Because the model was trained on low resolution faces.
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/P3.mp4" --audio "/content/sample_data/input_audio.wav" --resize_factor 2

In [ ]:


#@title 3.Use more padding to include the chin region (u can manually edit pads dimensions viewing and changing the code)
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/P3.mp4" --audio "/content/sample_data/input_audio.wav" --pads 0 20 0 0